In [1]:
import re
import os
import time
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from dotenv import load_dotenv
DOTENV_FILE = ".env"
load_dotenv(DOTENV_FILE, override=True)
import json
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../eval')
from utils import *


In [2]:
client = OpenAI()
label_mapping = {'accurate':1, 'inaccurate':0}
gpt_models = ['gpt-4o', 'gpt-4', 'gpt-4-turbo', 'gpt-3.5-turbo'] #'gpt-3.5-turbo', 

# Aggrefact prompt
system = '''Your task is to check if the Response is accurate to the Evidence.
Generate \'Accurate\' if the Response is accurate when verified according to the Evidence,
or \'Inaccurate\' if the Response is inaccurate (contradicts the evidence) or cannot be
verified.'''
user = '''**Evidence**\n\n{article}\n\n**End of Evidence**\n
**Response**:\n\n{summary}\n\n**End of Response**\n
Let's think step-by-step.'''

def call_gpt(system_prompt, user_prompt, model='gpt-4', temperature=0):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )
    result = completion.choices[0].message.content
    # print(result)
    result = re.sub(r'[^\w\s]', '', result)
    result = result.strip().lower().split()
    try:
        result = label_mapping[result[0]]
    except:
        # print(result)
        if "accurate" in result or "accurately" in result:
            result = 1
        elif "inaccurate" in result or "inaccurately" in result:
            result = 0
        else:
            result = None
            # raise ValueError
    return result

## Sample-level Prediction

In [3]:
# data_folder = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))), 'backup_data_with_detector_results')
# subfolders = [f.path for f in os.scandir(data_folder) if f.is_dir()]
# model_files = []
# for subfolder in subfolders:
#     model_files += [f.path for f in os.scandir(subfolder) if f.is_file()]
# print(model_files)
# selected_models = [
#     "openai/GPT-3.5-Turbo",
#     "openai/gpt-4o",
#     "Qwen/Qwen2.5-7B-Instruct",
#     "microsoft/Phi-3-mini-4k-instruct",
#     "cohere/command-r-08-2024",
#     "meta-llama/Meta-Llama-3.1-8B-Instruct",
#     "meta-llama/Meta-Llama-3.1-70B-Instruct",
#     "google/gemini-1.5-flash-001",
#     "Anthropic/claude-3-5-sonnet-20240620",
#     "mistralai/Mistral-7B-Instruct-v0.3",
# ]

# for idx, gpt_model in enumerate(gpt_models):
#     print(f"Run model {idx} - {gpt_model}")
#     for file_name in model_files:
#         print(file_name.replace(data_folder,'')[1:].replace('.csv',''))
#         if file_name.replace(data_folder,'')[1:].replace('.csv','') not in selected_models:
#             continue
#         with open(file_name) as f:
#             print(file_name)
#             df = pd.read_csv(file_name).fillna('')
#             if f"{gpt_model}-FACTSprompt" in df:
#                 continue
#             preds = []
#             for index, row in df.iterrows():
#                 # start_time = time.time()
#                 result = call_gpt(system, user.format(article=row['source'], summary=row['summary']), model=gpt_model)
#                 preds.append(result)
                    
#             df.insert(len(df.columns.tolist()), f"{gpt_model}-FACTSprompt", preds)
#             df.to_csv(file_name, mode='w', index=False, header=True)

In [ ]:
file_name = '../assign/examples_to_annotate.csv'
# df = pd.read_csv('../assign/examples_to_annotate.csv')
# for index, row in df.iterrows():
#     sources.append(row['source'])

for idx, gpt_model in enumerate(gpt_models):
    print(f"Run model {idx} - {gpt_model}")
    df = pd.read_csv(file_name).fillna('')
    if f"{gpt_model}-FACTSprompt" in df:
            continue
    preds = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        # start_time = time.time()
        result = call_gpt(system, user.format(article=row['source'], summary=row['summary']), model=gpt_model)
        preds.append(result)
                    
    df.insert(len(df.columns.tolist()), f"{gpt_model}-FACTSprompt", preds)
    df.to_csv(file_name, mode='w', index=False, header=True)

Run model 0 - gpt-4o


  0%|          | 5/1150 [00:14<49:50,  2.61s/it]  